In [56]:
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
import time

In [57]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
im = plt.imread("/content/drive/MyDrive/Colab Notebooks/image.jpg")

In [59]:
shape = np.shape(im)

In [60]:
imflatten = im.copy().reshape(shape[0]*shape[1],3)

In [61]:
(imflatten[0][0] + imflatten[0][1] + imflatten[0][2])/3

<ipython-input-61-67389afd88bb>:1: RuntimeWarning: overflow encountered in ubyte_scalars
  (imflatten[0][0] + imflatten[0][1] + imflatten[0][2])/3


52.666666666666664

In [62]:
shape2 = np.shape(imflatten)

In [63]:
t1 = time.time()
imtemp = np.empty_like(imflatten)

for i, pixel in enumerate(imflatten):
  gray = np.uint8((int(pixel[0]) + int(pixel[1]) + int(pixel[2]))/3)
  imtemp[i][0] = imtemp[i][1] = imtemp[i][2] = gray

t2 = time.time()

print(t2-t1)


3.92779278755188


In [64]:
imgray1 = imtemp.reshape(1080, 1920, 3)

In [65]:
from PIL import Image
imcpu = Image.fromarray(imgray1)
imcpu.save("/content/drive/MyDrive/Colab Notebooks/image_gray_CPU.jpeg")

In [66]:
cuda.is_available()

True

In [67]:
imflatten2 = im.copy().reshape(shape[0]*shape[1],3)

In [68]:
shape3 = np.shape(imflatten2)

In [69]:
devdata = cuda.to_device(imflatten2)
devOuput = cuda.device_array(shape3, np.uint8)

In [70]:
@cuda.jit
def grayscale(src, dst):
# where are we in the input?
  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  g = np.uint8((src[tidx, 0] + src[tidx, 1] + src[tidx, 2]) / 3)
  dst[tidx, 0] = dst[tidx, 1] = dst[tidx, 2] = g

In [71]:
t3 = time.time()

pixelCount = shape[0]*shape[1]
blockSize = 64
gridSize = int(pixelCount / blockSize)
grayscale[gridSize, blockSize](devdata, devOuput)

t4 = time.time()

print(t4-t3)


0.06766963005065918


In [72]:
imgray2 = devOuput.copy_to_host()
imgray3 = imgray2.reshape(1080, 1920, 3)
imgpu = Image.fromarray(imgray3)
imgpu.save("/content/drive/MyDrive/Colab Notebooks/image_gray_GPU.jpeg")

In [73]:
imgray3

array([[[138, 138, 138],
        [137, 137, 137],
        [134, 134, 134],
        ...,
        [165, 165, 165],
        [165, 165, 165],
        [166, 166, 166]],

       [[144, 144, 144],
        [143, 143, 143],
        [140, 140, 140],
        ...,
        [167, 167, 167],
        [167, 167, 167],
        [167, 167, 167]],

       [[138, 138, 138],
        [137, 137, 137],
        [136, 136, 136],
        ...,
        [166, 166, 166],
        [165, 165, 165],
        [167, 167, 167]],

       ...,

       [[ 41,  41,  41],
        [ 41,  41,  41],
        [ 40,  40,  40],
        ...,
        [ 39,  39,  39],
        [ 40,  40,  40],
        [ 40,  40,  40]],

       [[ 41,  41,  41],
        [ 41,  41,  41],
        [ 40,  40,  40],
        ...,
        [ 37,  37,  37],
        [ 38,  38,  38],
        [ 39,  39,  39]],

       [[ 41,  41,  41],
        [ 41,  41,  41],
        [ 40,  40,  40],
        ...,
        [ 37,  37,  37],
        [ 37,  37,  37],
        [ 37,  37,  37]]